## Write a Simple Backend

1. All backend should overwrite:
  - a class attribute `TARGET`: this is the target name of your backend
  - `default_config` class property
  - `apply` method, which is responsible for generating files or anything you want with the given `ugraph`

### The `Backend` interface

1. the class attribute `TARGET` should be overwriten and be of type `str`.
    - the value of `TARGET` should be unique or `ValueError` will be raised
2. one should overwrite the `default_config` class property
3. the first argument of `__init__` should be the config dictionary
    - this dictionary can be generated via a toml file (recommended)
    - this dictionary should be of the same format as the value returned by `default_config` class property

Here is a simple example:

In [1]:
from utensor_cgen.backend import Backend
from utensor_cgen.utils import class_property

class DummyBackend(Backend):
    TARGET = 'dummy-backend'

    def __init__(self, config):
        self.output_file = self.config[self.TARGET][self.COMPONENT]['output-file']

    def apply(self, ugraph):
        print('generating {}'.format(self.output_file), )
        with open(self.output_file, 'w') as fid:
            fid.write('#include <stdio.h>\n\n')
            fid.write('int main(int argc, char* argv[]) {\n')
            fid.write('    printf("graph name: {}\\n");\n'.format(ugraph.name))
            fid.write('    printf("ops in topological sorted order:\\n");\n')
            for op_name in ugraph.topo_order:
                fid.write('    printf("    {}\\n");\n'.format(op_name))
            fid.write('    return 0;\n}')

    @class_property
    def default_config(cls):
        """this default_config property will define the format of `config` value
        passed to `__init__`
        
        You should initialize your backend instance accordingly
        """
        return {
            cls.TARGET: {
                    cls.COMPONENT: {
                    'output-file': 'list_op.c'
                }
            }
        }


In [2]:
print(DummyBackend.TARGET, DummyBackend.COMPONENT)

dummy-backend backend


Basically, this dummy backend takes a graph and generate a `.c` file which will print out all the operators in the graph.

## Backend Registration

Once you create a `Backend`, you can registrate it via `BackendManager` as following:

In [3]:
from utensor_cgen.backend import BackendManager

BackendManager.register(DummyBackend)

__main__.DummyBackend

## Read a Graph and Apply the Backend

In [4]:
from utensor_cgen.frontend import FrontendSelector

In [5]:
ugraph = FrontendSelector.parse('models/cifar10_cnn.pb', output_nodes=['pred'])

In [6]:
BackendManager.get_backend(DummyBackend.TARGET)({
    DummyBackend.TARGET: {
        'backend': {
            'output-file': 'list_mymodel_ops.cpp'
        }
    }
}).apply(ugraph)

generating list_mymodel_ops.cpp


In [7]:
!cat list_mymodel_ops.cpp

#include <stdio.h>

int main(int argc, char* argv[]) {
    printf("graph name: cifar10_cnn\n");
    printf("ops in topological sorted order:\n");
    printf("    Placeholder\n");
    printf("    conv/Variable\n");
    printf("    conv/feature_map\n");
    printf("    conv/bias\n");
    printf("    conv/logits\n");
    printf("    conv_1/Variable\n");
    printf("    conv_1/feature_map\n");
    printf("    conv_1/bias\n");
    printf("    conv_1/logits\n");
    printf("    conv_1/activation\n");
    printf("    MaxPool\n");
    printf("    conv_2/Variable\n");
    printf("    conv_2/feature_map\n");
    printf("    conv_2/bias\n");
    printf("    conv_2/logits\n");
    printf("    conv_3/Variable\n");
    printf("    conv_3/feature_map\n");
    printf("    conv_3/bias\n");
    printf("    conv_3/logits\n");
    printf("    conv_3/activation\n");
    printf("    MaxPool_1\n");
    printf("    conv_4/Variable\n");
    printf("    conv_4/feature_map\n");
    printf("    conv_4/bias\n");
 